In [1]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
import os
print(os.getenv("OPENAI_API_KEY"))

In [3]:
%%writefile app.py
import os
import streamlit as st
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
from symspellpy import SymSpell, Verbosity
from openai import OpenAI

# 🔹 Configure paths
poppler_path = r"C:\\Program Files\\poppler-24.08.0\\Library\\bin"
tesseract_path = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = tesseract_path

# 🔹 Load SymSpell for spelling correction
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = r"D:\\space kids india\\en-80k.txt"

if not sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1):
    st.error("❌ Dictionary file not found! Check the file path.")
    st.stop()

# 🔹 Ensure API key is securely set
API_KEY = os.getenv("OPENAI_API_KEY")
if not API_KEY:
    st.error("❌ API Key not found. Set it using os.environ['OPENAI_API_KEY'] = 'your_api_key'")
    st.stop()

# 🔹 Initialize OpenAI client for OpenRouter
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=API_KEY,
)

# 🔹 Function to correct spelling
def correct_spelling(text):
    words = text.split()
    corrected_words = [
        sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)[0].term 
        if sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2) 
        else word 
        for word in words
    ]
    return " ".join(corrected_words)

# 🔹 Function to extract text from image
def extract_text_from_image(image):
    try:
        text = pytesseract.image_to_string(image, lang='eng')
        corrected_text = correct_spelling(text)
        return corrected_text
    except Exception as e:
        st.error(f"❌ Error extracting text: {e}")
        return ""

# 🔹 Function to extract legal keywords
def extract_legal_keywords(text):
    important_keywords = ["complaint", "corruption", "vigilance", "harassment", "police"]
    keywords = [word for word in text.split() if word.lower() in important_keywords]
    return ", ".join(set(keywords))  # Remove duplicates

# 🔹 Function to query AI for legal insights
def chat_with_deepseek(prompt):
    try:
        completion = client.chat.completions.create(
            model="deepseek/deepseek-r1-zero:free",  
            messages=[
                {"role": "system", "content": "You are an AI legal assistant providing case insights and legal information."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        return completion.choices[0].message.content if completion.choices else "⚠️ No response from AI."
    except Exception as e:
        return f"❌ Error: {str(e)}"

# --- STREAMLIT UI ---
st.title("📜 Legal Document Analyzer")

uploaded_file = st.file_uploader("📂 Upload an Image", type=["png", "jpg", "jpeg", "bmp", "tiff", "gif"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Extract text
    extracted_text = extract_text_from_image(image)
    if extracted_text.strip():
        st.subheader("📄 Extracted Text:")
        st.text_area("Extracted Text", extracted_text, height=200)

        # Extract keywords
        keywords = extract_legal_keywords(extracted_text)
        if keywords:
            st.subheader("🔍 Legal Keywords Found:")
            st.write(keywords)

            legal_details = chat_with_deepseek(f"Provide legal details and solutions for: {keywords}")
            st.subheader("⚖️ Legal Insights:")
            st.write(legal_details)
        else:
            st.warning("⚠️ No relevant legal keywords found.")
    else:
        st.warning("⚠️ No text detected. Try a clearer image.")

# Chatbot Section
st.subheader("💬 Legal Chatbot")
user_query = st.text_input("Ask a legal question:")
if st.button("Ask"):
    if user_query.strip():
        response = chat_with_deepseek(user_query)
        st.write("Bot:", response)
    else:
        st.warning("⚠️ Please enter a question.")



Overwriting app.py


In [ ]:
!streamlit run app.py